In [ ]:
import tensorflow as tf
import os
import pickle
import numpy as np

In [ ]:
CIFAR_DIR = '/home/commaai-03/Data/dataset/cifar-10-python'

def unpickle(file):
    import pickle
    with open(file, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    return data

In [ ]:
filenames = [os.path.join(CIFAR_DIR, file) 
             for file in os.listdir(CIFAR_DIR)
             if '.html' not in file]
filenames.sort()
meta_file = filenames[0]
train_files = filenames[1:-1]
test_file = [filenames[-1]]

In [ ]:
_test_data = unpickle(test_file[0])
for k, v in _test_data.items():
    print(k)

In [ ]:
img_arr = _test_data[b'data'][0]
# 32 * 32 * 3 (R,G,B)
img_arr_reshaped = img_arr.reshape((3, 32, 32))
img = img_arr_reshaped.transpose(1, 2, 0)
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline
imshow(img)

In [ ]:
class CifarData:
    
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = self.load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print('[CIFAR-10]: Data shape-> %s' % str(self._data.shape))
        print('[CIFAR-10]: Label shape-> %s' % str(self._labels.shape))
        
        self.num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def load_data(self, filename):
        import pickle
        with open(filename, mode='rb') as f:
            data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
    def _shuffle_data(self):
        index = np.random.permutation(self.num_examples)
        self._data = self._data[index]
        self._labels = self._labels[index]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self.num_examples:
            rest_num_examples = self.num_examples - self._indicator
            data_rest_part = self._data[self._indicator: self.num_examples]
            label_rest_part = self._labels[self._indicator: self.num_examples]
            
            if self._need_shuffle:
                self._shuffle_data()
            # For new loop, self._indicator + batch_size = self.num_examples
            self._indicator = batch_size - rest_num_examples
            end_indicator = self._indicator
            data_new_part = self._data[:end_indicator]
            label_new_part = self._labels[:end_indicator]
            batch_data = np.concatenate((data_rest_part, data_new_part), axis=0)
            batch_label = np.concatenate((label_rest_part, label_new_part), axis=0)
        else:
            batch_data = self._data[self._indicator:end_indicator]
            batch_label = self._labels[self._indicator:end_indicator]
            self._indicator = end_indicator
        
        return batch_data, batch_label

In [ ]:
train_data = CifarData(train_files, need_shuffle=True)
test_data = CifarData(test_file, need_shuffle=False)

In [ ]:
def residual_block(x, output_channel):
    '''
    Residual connection implementation.
    '''
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1,1)
    else:
        raise Exception('Input channel can\'t match output channel.')
    conv1 = tf.keras.layers.Conv2D(filters=output_channel, 
                                   kernel_size=(3,3), 
                                   strides=strides,
                                   padding='same', 
                                   activation=tf.nn.relu, 
                                   name='conv1')(x)
    print('conv1->shape:', conv1.get_shape())
    conv2 = tf.keras.layers.Conv2D(filters=output_channel, 
                                   kernel_size=(3,3), 
                                   strides=(1,1), 
                                   padding='same', 
                                   activation=tf.nn.relu, 
                                   name='conv2')(conv1)
    print('conv2->shape:', conv2.get_shape())
    
    if increase_dim:
        pooled_x = tf.keras.layers.AveragePooling2D(pool_size=(2,2), 
                                    strides=(2,2), 
                                    padding='valid')(x)
        padded_x = tf.pad(pooled_x, [[0,0], 
                                     [0,0],
                                     [0,0],
                                     [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    
    output_x = conv2 + padded_x
    return output_x

def res_net(x, 
            num_residual_blocks,
            num_filter_base,
            class_num):
    '''
    Args:
    - x:
    - num_residual_blocks: eg: [3, 4, 6, 3]
    - num_filter_base:
    - class_num:
    '''
    num_subsampling = len(num_residual_blocks)
    layers = []
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.keras.layers.Conv2D(filters=num_filter_base, 
                                       kernel_size=(3,3), 
                                       strides=(1,1), 
                                       padding='same', 
                                       activation=tf.nn.relu, 
                                       name='conv0')(x)
        layers.append(conv0)
    # eg: num_subsampling = 4, sample_id = [0, 1, 2, 3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope('conv%d_%d' % (sample_id, i)):
                print('sample_id: %d, i: %d' % (sample_id, i))
                conv = residual_block(x=layers[-1], 
                    output_channel=num_filter_base * (2 ** sample_id))
                layers.append(conv)
    
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]
    with tf.variable_scope('fc'):
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logits = tf.keras.layers.Dense(class_num)(global_pool)
        layers.append(logits)
    return layers[-1]

In [ ]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_img = tf.reshape(x, [-1, 3, 32, 32])
x_img = tf.transpose(x_img, perm=[0, 2, 3, 1])

y_ = res_net(x=x_img,
             num_residual_blocks=[2,3,2], 
             num_filter_base=32, 
             class_num=10)

_loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(_loss)

In [ ]:
init = tf.global_variables_initializer()
batch_size = 64
train_steps = 100000
test_steps = 2000

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss, acc, _ = sess.run([_loss, accuracy, train_op], 
                                feed_dict={x: batch_data, y: batch_labels})
        
        if (i+1) % 100 == 0:
            print('[Train]: Step: %d, loss: %4.5f acc: %4.5f' 
                  % (i+1, loss, acc))
        if (i+1) % 5000 == 0:
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))